In [1]:
from api import spotify_api
import json
from datetime import datetime, timedelta
import re
from time import gmtime, strftime
import git
import requests
# https://gitpython.readthedocs.io/en/stable/intro.html

settings_file = './api/settings.txt'
albums_cache_file = './albums_cache.txt'

max_album_age = 4 #years

In [2]:
access_token = spotify_api.refresh_token(settings_file)


Requesting refresh: 
<Response [200]>
200 OK
BQAaiTyhOVJP6lo6i9RqlvT4Bu70uA6oeIIo7cXFekqevVgw6Na3_Fqsvwh4fIHrcfD3e3SCU-LSgukj8XmHyszv0kBPBVaKn1VfHUitbc9xCp3MEfgB2Bl7OVP1S-_6GPgrUdOVP1QRUKsuDGxq-KB3Qw8POAQFWTt1Ijc2x-5nVPepIfT3YWrGN6zsUR38VUbra9AaOMCNhAOoOU7WI5jZKsyMb8WP6sAzlN8_WW_ouzky8FUgAca-bYc67bWMeppp_0b1BQEKLtTTDO5P0zNnm-l_EeC1
BQAaiTyhOVJP6lo6i9RqlvT4Bu70uA6oeIIo7cXFekqevVgw6Na3_Fqsvwh4fIHrcfD3e3SCU-LSgukj8XmHyszv0kBPBVaKn1VfHUitbc9xCp3MEfgB2Bl7OVP1S-_6GPgrUdOVP1QRUKsuDGxq-KB3Qw8POAQFWTt1Ijc2x-5nVPepIfT3YWrGN6zsUR38VUbra9AaOMCNhAOoOU7WI5jZKsyMb8WP6sAzlN8_WW_ouzky8FUgAca-bYc67bWMeppp_0b1BQEKLtTTDO5P0zNnm-l_EeC1


In [3]:
last_success_file = "/Users/bednar/Logs/spotify_news_last_success.txt"
datetime_format = "%Y-%m-%d %H:%M:%S"

f = open(last_success_file, "r")
last_success_datetime = datetime.strptime(f.read(), '%Y-%m-%d %H:%M:%S')
print(f"Last success: {last_success_datetime}")

Last success: 2023-10-01 08:00:00


In [4]:
SPOTIFY_API_URL = 'https://api.spotify.com/v1'
#AUTH_URL = 'https://accounts.spotify.com/api/token'

def get_recent_tracks(token, artist_id, release_date):
    """
    Get recent tracks for a given artist ID and release date.
    """
    headers = {
        'Authorization': f'Bearer {token}',
    }

    response = requests.get(f'{SPOTIFY_API_URL}/artists/{artist_id}/albums', headers=headers)
    tracks = []

    for album in response.json()['items']:
        album_release_date = (album['release_date'] + "-01-01")[:10]
        album_release_date = datetime.strptime(album_release_date, '%Y-%m-%d')
        if album_release_date > release_date:
            album_tracks_response = requests.get(f'{SPOTIFY_API_URL}/albums/{album["id"]}/tracks?limit=50', headers=headers)
            tracks.extend(album_tracks_response.json()['items'])

    return tracks

In [5]:
def get_liked_artists(token, verbose = False):
    spotify_request_url = 'https://api.spotify.com/v1/'

    next_request = spotify_request_url + 'me/following?type=artist&limit=10'
    artists_dict = {}

    while(True):
        next_request = next_request + '&access_token=' + token.strip()
        artists_json = spotify_api.spotify_request(next_request)

        if verbose:
            print('\n')
        artist_items = artists_json['artists']['items']
        for item in artist_items:
            if verbose:
                print(item['name']+' -> '+item['id'])
            artists_dict[item['name']] = item['id']

        next_request = artists_json['artists']['next']

        if verbose:
            print('\n')
        if not next_request:
            if verbose:
                print("---END---")
            break
            
    return artists_dict

In [6]:


# Get liked artists
#liked_artists_response = requests.get(f'{SPOTIFY_API_URL}/me/following?type=artist', headers={'Authorization': f'Bearer {access_token}'})
#liked_artists = liked_artists_response.json()['artists']['items']
liked_artists = get_liked_artists(access_token, False)

# Get recent tracks for each liked artist
all_recent_tracks = []
for artist in liked_artists.keys():
    artist_recent_tracks = get_recent_tracks(access_token, liked_artists[artist], last_success_datetime)
    all_recent_tracks.extend(artist_recent_tracks)

# Extract track URIs
track_uris = [track['uri'] for track in all_recent_tracks]

len(track_uris)
# 57

178

In [7]:
headers = {
    'Authorization': f'Bearer {access_token.strip()}',
}

response = requests.get(f'{SPOTIFY_API_URL}/me/playlists', headers=headers)
playlists = response.json()['items']

for pl in playlists:
    print(f"{pl['id']} : {pl['name']}")

3X7d5f6ghRbZ6yp7EfWfNv : Temp
37i9dQZF1DWTXGqmP0bfT3 : Energy Booster: Metal
37i9dQZF1F0sijgNaJdgit : Your Top Songs 2022
7fbLjIxIe8TpcDNzJL74Vo : Nu Metal <3
0cobLoG4h5dlwzUOtUvhhu : Worthy Oldies
37i9dQZF1DWY5ai7gxfuaS : Alternative Metal
37i9dQZF1EJBYilw4SIU9l : Anna + piotr
21bOfrCkaja2QNZbrDpOWA : Garden Party
1dIKRltCE0sSsdNyDgEFxA : Kiedy starej nie ma
5QtunQsOyazjCStL8KNz3U : Music News (Bednar)
1SXZgziRBTJwU3wy1HQjJs : Discoveries!
3IG5YeyDxr2WvmtxoABXco : Cheerful raps
7dxKj0tjReCC9lh9dho2bt : Hard Pop
7sOohuBQejSZPQBIkvmh1C : Make it right
5UWkugaExwjDzZ88TQxkKv : Gimbaza
37i9dQZF1E36nvkKffTErV : Daily Mix 1
37i9dQZF1E38Fn9m81W7RQ : Daily Mix 2
37i9dQZF1E3667zvRrCgGf : Daily Mix 3
37i9dQZF1E37v344O1UgHe : Daily Mix 4
37i9dQZF1E39mI1lvzAmDP : Daily Mix 5


In [8]:
max_tracks_per_request = 10
my_playlist = '5QtunQsOyazjCStL8KNz3U'
url = 'https://api.spotify.com/v1/users/piotr.bednarski/playlists/' + my_playlist + '/tracks?uris='
headers = {
    'Authorization' : 'Bearer ' + access_token.strip(),
    'Content-Type' : 'application/json'
}
url_full = url
for i in range(len(track_uris)):
    #print(track_uris[i])
    url_full = url_full + track_uris[i] + ','
    
    if (i%max_tracks_per_request == max_tracks_per_request-1) or (i == len(track_uris)-1):
        url_full = url_full[:-1]
        #print("Request!")
        r = requests.post(url_full, headers=headers)
        #print(r)
        print(r.status_code, r.reason)
        url_full = url

201 Created
201 Created
201 Created
201 Created
201 Created
201 Created
201 Created
201 Created
201 Created
201 Created
201 Created
201 Created
201 Created
201 Created
201 Created
201 Created
201 Created
201 Created


In [9]:
# Save last success datetime

last_success_datetime = datetime.now()

f = open(last_success_file, "w")
f.write(last_success_datetime.strftime(datetime_format))
f.close()

print(f"New success date: {last_success_datetime}")

New success date: 2023-10-25 12:17:23.615296


In [10]:
stop here

SyntaxError: invalid syntax (<ipython-input-10-a96ba3aab008>, line 1)

In [ ]:
next_request = 'https://api.spotify.com/v1/me/playlists?access_token=' + access_token.strip()
playlists_json = spotify_api.spotify_request(next_request)

my_playlist = '5QtunQsOyazjCStL8KNz3U'

playlists_items = playlists_json['items']
for item in playlists_items:
    print(item['name'] + ' -> ' + item['id'])
    if item['name'].strip() == 'Music News (Bednar)':
        my_playlist = item['id'].strip()
        
print('\nMy playlist ID: ' + my_playlist)

In [ ]:
import requests

max_tracks_per_request = 10
url = 'https://api.spotify.com/v1/users/piotr.bednarski/playlists/' + my_playlist + '/tracks?uris='
headers = {
    'Authorization' : 'Bearer ' + access_token.strip(),
    'Content-Type' : 'application/json'
}
url_full = url
for i in range(len(new_tracks)):
    #print(new_tracks[i])
    url_full = url_full + new_tracks[i] + ','
    
    if (i%max_tracks_per_request == max_tracks_per_request-1) or (i == len(new_tracks)-1):
        url_full = url_full[:-1]
        #print("Request!")
        r = requests.post(url_full, headers=headers)
        #print(r)
        print(r.status_code, r.reason)
        url_full = url

In [ ]:
with open('feed_stub.xml', 'r') as fs:
    feed_stub=fs.read()

In [ ]:
title = strftime("%Y-%m-%d %H:%M Spotify newest releases", gmtime())
feed = re.sub('TITLE', title, feed_stub, flags=(re.MULTILINE | re.DOTALL))

In [ ]:
description = 'New releases:&lt;br&gt;'
for item in new_albums_dict:
    description = description + '&lt;br&gt;' + new_albums_dict[item][0]+ ' - ' + new_albums_dict[item][1]
print(description)
feed = re.sub('DESCRIPTION', description, feed, flags=(re.MULTILINE | re.DOTALL))

In [ ]:
with open('feed.xml', 'w', encoding='utf-8') as f:
    f.write(feed)

In [ ]:
repo = git.Repo( '/home/piotr.bednarski/Repositories/spotify-new-releases' )
print(repo.git.add( '.' ))

timestring = strftime("%Y%m%d_%H%M%S", gmtime())
message = '"Update at ' + timestring + '"'

print(repo.git.commit( m=message ))
print(repo.git.push())
print(repo.git.status())

In [ ]:
dict(beam_job_id="test")